In [7]:
import urllib
import requests
from bs4 import BeautifulSoup
import re

In [8]:
user='VandalBot'
passw='Goawaybadbots'
baseurl='http://wikipast.epfl.ch/wikipast/'
summary='Lutte activement contre les nombreux SPAMBot qui assiègent Wikipast.'

In [9]:
# Login request
payload={'action':'query','format':'json','utf8':'','meta':'tokens','type':'login'}
r1=requests.post(baseurl + 'api.php', data=payload)

#login confirm
login_token=r1.json()['query']['tokens']['logintoken']
payload={'action':'login','format':'json','utf8':'','lgname':user,'lgpassword':passw,'lgtoken':login_token}
r2=requests.post(baseurl + 'api.php', data=payload, cookies=r1.cookies)

In [10]:
#get edit token2
params3='?format=json&action=query&meta=tokens&continue='
r3=requests.get(baseurl + 'api.php' + params3, cookies=r2.cookies)
edit_token=r3.json()['query']['tokens']['csrftoken']

edit_cookie=r2.cookies.copy()
edit_cookie.update(r3.cookies)

In [17]:
fullbios = ''
name = 'Biographies'
result=requests.post(baseurl+'api.php?action=query&titles='+name+'&export&exportnowrap')
soup=BeautifulSoup(result.text, "lxml")
code=''
for primitive in soup.findAll("text"):
    code+=primitive.string
fullbios = code

In [18]:
users2018 = open("liste_utilisateurs_2018.txt", "r")
lines = users2018.readlines()
list_users_2018 = [l.strip('\n') for l in lines]

In [19]:
users2017 = open("liste_utilisateurs_2017.txt", "r")
lines = users2017.readlines()
list_users_2017 = [l.strip('\n') for l in lines]

In [107]:
goodBots = open("list_good_bots.txt", "r")
lines = goodBots.readlines()
list_good_bots = [l.strip('\n') for l in lines]

In [129]:
protected_logins = list(set(list_users_2018).union(set(list_users_2017)))

In [130]:
def getPageList(logins):
    liste_pages=[]
    for user in logins:
        result=requests.post(baseurl+'api.php?action=query&list=usercontribs&ucuser='+user+'&format=xml')
        soup=BeautifulSoup(result.content,'lxml')
        for primitive in soup.usercontribs.findAll('item'):
            liste_pages.append(primitive['title'])
    liste_pages=list(set(liste_pages))
    return liste_pages

In [131]:
humansPages = getPageList(protected_logins)

In [132]:
protected_logins = list(set(list_users_2018).union(set(list_users_2017)).union(set(list_good_bots)))

In [133]:
def getRandomPage():
    result=requests.post(baseurl+'index.php/Special:Page_au_hasard')
    return result.url[43:]

In [ ]:
botsPages = []
while(len(botsPages) < len(humansPages)):
    tmp = getRandomPage()
    if((tmp not in humansPages) and (tmp not in botsPages)):
        botsPages.append(tmp)

In [ ]:
humansPages

In [ ]:
botsPages

In [ ]:
humansTextPages = []
botsTextPages = []
for p in humansPages:
    result = requests.post(baseurl+'index.php/'+p)
    